# AI agent

## 개발환경 설정

### 파이썬 개발환경 테스트

In [ ]:
1 + 1

### OPENAI API

In [ ]:
api_key = 'sk------'

### 마운트 설정: 구글 코랩만 실행

In [ ]:
# 1. 마운트 설정
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 2. 현재 작업디렉터리 연결
import os
os.chdir("drive/My Drive/app_ai_agent/")

In [ ]:
# 3. 작업디렉터리가 가진 목록을 화면에 표시
!ls

In [ ]:
# 작업디렉터리 파일을 읽는 예제 실행
with open('bearlist.txt', 'r') as f:
    for line in f:
        print(line)  # print(line, end='')

## API VS 라이브러리

#### 음성을 텍스트로 변환시: whisper라이브러리 설치 후 사용

In [ ]:
!pip install -q openai-whisper ffmpeg

In [ ]:
# 3. Whisper 불러오기
import whisper

# 4. 모델 로드 (base 권장, 더 빠름 / 정확도 높이려면 'small' 또는 'medium')
model = whisper.load_model("base")

# 5. 오디오 파일 업로드
from google.colab import files
uploaded = files.upload()

# 업로드한 파일 이름 가져오기
import os
audio_file = list(uploaded.keys())[0]

# 6. 음성 파일을 텍스트로 변환
result = model.transcribe(audio_file, language="ko")

# 7. 텍스트 출력
print("\n===== 추출된 텍스트 =====\n")
print(result["text"])


In [ ]:
# 8. txt 파일로 저장
output_path = "chap05/audio/" + audio_file.replace(".wav", ".txt").replace(".mp4", ".txt").replace(".mp3", ".txt")
with open(output_path, "w", encoding="utf-8") as f:
    f.write(result["text"])

print(f"\n✅ 변환 완료: {output_path}")

# 9. 다운로드 링크 제공
from google.colab import files
files.download(output_path)

#### 음성을 텍스트로 변환시: API 사용- OPENAI 서버사용

In [ ]:
# OPENAI API키 인증
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()
# api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key=api_key)

In [ ]:
# 오디오 파일 업로드
from google.colab import files
uploaded = files.upload()

# 업로드한 파일 이름 가져오기
import os
audio_file = list(uploaded.keys())[0]

with open(audio_file, 'rb') as audio_file:
    transcription = client.audio.transcriptions.create(  # OPENAI의 서버로 전송해서 음성을 텍스트로 변환
        model="whisper-1",  # 변환에 사용할 모델
        file=audio_file  # 변환할 음성파일
    )

transcription  # 음성을 텍스트로 변환한 결과

In [ ]:
# txt 파일로 저장
output_path = "chap05/audio/" + audio_file.name.replace(".wav", ".txt").replace(".mp4", ".txt").replace(".mp3", ".txt")
with open(output_path, "w", encoding="utf-8") as f:
    f.write(transcription.text)

print(f"\n✅ 변환 완료: {output_path}")

# 9. 다운로드 링크 제공
from google.colab import files
files.download(output_path)

## 6장

In [ ]:
# 123p, image_explanation.ipynb
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()
# api_key = os.getenv("OPENAI_API_KEY")  # 환경 변수에서 API 키를 가져오기

client = OpenAI(api_key=api_key)  # 오픈AI 클라이언트의 인스턴스 생성

messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "이 이미지에 대해 설명해주세요."},
            {
                "type": "image_url",
                "image_url": {
                    "url": "https://images.unsplash.com/photo-1736264335247-8ec5664c8328?q=80&w=1887&auto=format&fit=crop&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D",
                },
            },
        ],
    }
]

response = client.chat.completions.create(
    model="gpt-4o",  # 응답 생성에 사용할 모델 지정
    messages=messages # 대화 기록을 입력으로 전달
)

response

In [ ]:
# 124p, image_explanation.ipynb
# 이미지를 base64로 변환하기
import base64

# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

image_path = "chap06/data/images/mangwon_bakery.jpg"

# 이미지를 base64로 인코딩
base64_image = encode_image(image_path)

print(base64_image)

In [ ]:
# 123p, image_explanation.ipynb
# base64로 변환한 이미지 설명 요청하기
messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "이 이미지에 대해 설명해주세요."},
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{base64_image}",
                },
            },
        ],
    }
]

response = client.chat.completions.create(
    model="gpt-4o",  # 응답 생성에 사용할 모델 지정
    messages=messages # 대화 기록을 입력으로 전달
)

response.choices[0].message.content

In [ ]:
# 127p, image_explanation.ipynb
# 여러 이미지 설명 요청하기

seolleung_terrarosa_base64 = encode_image("chap06/data/images/seolleung_terrarosa.jpg")
local_stitch_terrarosa_base64 = encode_image("chap06/data/images/local_stitch_terrarosa.jpg")

messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "두 카페의 차이점을 설명해주세요."},
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{seolleung_terrarosa_base64}",
                },
            },
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{local_stitch_terrarosa_base64}",
                },
            },
        ],
    }
]

response = client.chat.completions.create(
    model="gpt-4o",  # 응답 생성에 사용할 모델 지정
    messages=messages # 대화 기록을 입력으로 전달
)

response.choices[0].message.content

In [ ]:
# 130p, image_explanation.ipynb
# 연구 개발비 비교하기
oecd_rnd_2021_base64 = encode_image("chap06/data/images/oecd_rnd_2021_large.png")
oecd_rnd_2022_base64 = encode_image("chap06/data/images/oecd_rnd_2022_large.png")

messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "첫번째는 2021년 데이터이고, 두번째는 2022년 데이터입니다. 이 데이터에 대해 설명해주세요. 어떤 변화가 있었나요? 한국 중심으로 설명해주세요."},
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{oecd_rnd_2021_base64}",
                },
            },
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{oecd_rnd_2022_base64}",
                },
            },
        ],
    }
]

response = client.chat.completions.create(
    model="gpt-4o",  # 응답 생성에 사용할 모델 지정
    messages=messages # 대화 기록을 입력으로 전달
)

response.choices[0].message.content


In [ ]:
# 131p, image_explanation.ipynb
# 연구 개발비 비교하기
oecd_rnd_2021_base64 = encode_image("chap06/data/images/oecd_rnd_2021_medium.png")
oecd_rnd_2022_base64 = encode_image("chap06/data/images/oecd_rnd_2022.png")

messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "첫번째는 2021년 데이터이고, 두번째는 2022년 데이터입니다. 이 데이터에 대해 설명해주세요. 어떤 변화가 있었나요? 한국 중심으로 설명해주세요."},
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{oecd_rnd_2021_base64}",
                },
            },
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{oecd_rnd_2022_base64}",
                },
            },
        ],
    }
]

response = client.chat.completions.create(
    model="gpt-4o",  # 응답 생성에 사용할 모델 지정
    messages=messages # 대화 기록을 입력으로 전달
)

response.choices[0].message.content

In [ ]:
# 133p, image_quiz.py
# 연구 개발비 비교하기
from glob import glob
import json
from openai import OpenAI
from dotenv import load_dotenv
import os
import base64

load_dotenv()
# api_key = os.getenv("OPENAI_API_KEY")  # 환경 변수에서 API 키 가져오기
client = OpenAI(api_key=api_key)  # OpenAI 클라이언트의 인스턴스 생성


In [ ]:
# 이미지 경로를 받아서 base64로 인코딩하는 함수
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

In [ ]:
# 이미지 경로를 받아서 퀴즈를 만드는 함수
def image_quiz(image_path, n_trial=0, max_trial=3):
    if n_trial >= max_trial: # 최대 시도 회수에 도달하면 포기
        raise Exception("Failed to generate a quiz.")

    base64_image = encode_image(image_path) # 이미지를 base64로 인코딩

    quiz_prompt = """
    제공된 이미지를 바탕으로, 다음과 같은 양식으로 퀴즈를 만들어주세요.
    정답은 1~4 중 하나만 해당하도록 출제하세요.
    토익 리스닝 문제 스타일로 문제를 만들어주세요.
    아래는 예시입니다.
    ----- 예시 -----

    Q: 다음 이미지에 대한 설명 중 옳지 않은 것은 무엇인가요?
    - (1) 베이커리에서 사람들이 빵을 사고 있는 모습이 담겨 있습니다.
    - (2) 맨 앞에 서 있는 사람은 빨간색 셔츠를 입고 있습니다.
    - (3) 기차를 타기 위해 줄을 서 있는 사람들이 있습니다.
    - (4) 점원은 노란색 티셔츠를 입고 있습니다.

    Listening: Which of the following descriptions of the image is incorrect?
    - (1) It shows people buying bread at a bakery.
    - (2) The person standing at the front is wearing a red shirt.
    - (3) There are people lining up to take a train.
    - (4) The clerk is wearing a yellow T-shirt.

    정답: (4) 점원은 노란색 티셔츠가 아닌 파란색 티셔츠를 입고 있습니다.
    (주의: 정답은 1~4 중 하나만 선택되도록 출제하세요.)
    ======
    """

    messages = [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": quiz_prompt},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{base64_image}",
                    },
                },
            ],
        }
    ]

    try:
        response = client.chat.completions.create(
            model="gpt-4o",  # 응답 생성에 사용할 모델 지정
            messages=messages # 대화 기록을 입력으로 전달
        )
    except Exception as e:
        print("failed\n" + e)
        return image_quiz(image_path, n_trial+1)

    content = response.choices[0].message.content

    if "Listening:" in content:
        return content, True
    else:
        return image_quiz(image_path, n_trial+1)


In [ ]:
# 이미지 경로를 받아서 퀴즈를 만듦, 135p
q = image_quiz("chap06/data/images/busan_dive.jpg")
print(q)

In [ ]:
# 여러 이미지로 문제집 만들기, 135p
txt = '' # 문제들을 계속 붙여 나가기 위해 빈 문자열 선언
eng_dict = []
no = 1 # 문제 번호를 위해 선언
for g in glob('chap06/data/images/*.jpg'):  # ②
    q, is_suceed = image_quiz(g)

    if not is_suceed:
        continue


    divider = f'## 문제 {no}\n\n'
    print(divider)

    txt += divider
    # 파일명 추출해 이미지 링크 만들기
    filename = os.path.basename(g) # 마크다운에 표시할 이미지 파일 경로 설정
    txt += f'![image]({filename})\n\n'

    # 문제 추가
    print(q)
    txt += q + '\n\n---------------------\n\n'
    # 마크다운 파일로 저장
    with open('chap06/data/images/image_quiz_eng.md', 'w', encoding='utf-8') as f:
        f.write(txt)

    # 영어 문제만 추출
    eng = q.split('Listening: ')[1].split('정답:')[0].strip()

    eng_dict.append({
        'no': no,
        'eng': eng,
        'img': filename
    })

    # json 파일로 저장
    with open('chap06/data/images/image_quiz_eng.json', 'w', encoding='utf-8') as f:
        json.dump(eng_dict, f, ensure_ascii=False, indent=4)


    no += 1 # 문제 번호 증가



In [ ]:
# 143p
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()
# api_key = os.getenv("OPENAI_API_KEY")  # 환경 변수에서 API 키를 가져옵니다.
client = OpenAI(api_key=api_key)  # OpenAI 클라이언트의 인스턴스를 생성합니다.

In [ ]:
# 첫번째 TTS테스트, 143p
response = client.audio.speech.create(
    model="tts-1-hd",
    voice="alloy",
    input="Hello world! This is a TTS test.",
)

response.write_to_file("data/hello_world.mp3")

# 재생
import IPython.display as ipd

ipd.Audio("data/hello_world.mp3")

In [ ]:
# 목소리를 ash로 바꾸고 TTS테스트, 144p
# 다른 목소리
voice = "ash"
mp3_file = "data/" + f"hello_world_{voice}.mp3"

response = client.audio.speech.create(
    model="tts-1-hd",
    voice=voice,
    input=f"Hello world! I'm {voice}. This is a TTS test.",
)

response.write_to_file(mp3_file)

# 재생
import IPython.display as ipd

ipd.Audio(mp3_file)

In [ ]:
# 영어스크립트 JSON파일 읽기, 145p
# JSON 로드
import json

# json 파일 열기
with open('chap06/data/images/image_quiz_eng.json', 'r', encoding='utf-8') as f:
    eng_dict = json.load(f)

eng_dict

In [ ]:
# 영어스크립트 JSON파일 읽기, 145p
# 로드된 JSON TTS로 읽기 파일 생성
voices = ['alloy', 'ash', 'coral', 'echo', 'fable', 'onyx', 'nova', 'sage' , 'shimmer']

for q in eng_dict:
    no = q['no']
    quiz = q['eng']
    quiz = quiz.replace("- (1)", "- One.\t")
    quiz = quiz.replace("- (2)", "- Two.\t")
    quiz = quiz.replace("- (3)", "- Three.\t")
    quiz = quiz.replace("- (4)", "- Four.\t")

    print(no, quiz)

    voice = voices[no % len(voices)] # 문제 개수를 목소리 개수로 나눈 나머지 값으로 선택

    response = client.audio.speech.create(
        model="tts-1-hd",
        voice=voice,
        input=f'#{no}. {quiz}',
    )

    response.write_to_file(f"chap06/data/audio/{no}.mp3")


In [ ]:
# 영어스크립트 JSON파일 읽기, 146p
# 생성된 TTS로 오디오 파일 재생
ipd.Audio(f"chap06/data/audio/1.mp3")

## 7장

In [ ]:
# 현재 날짜와 시간을 알려주는 함수 정의, 149p
from datetime import datetime

def get_current_time():
    now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(now)
    return now

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_time",
            "description": "현재 날짜와 시간을 반환합니다.",
        }
    },
]


if __name__ == '__main__':
    get_current_time()

In [ ]:
# GPT에 tools정보 포함하기, 151p
# from gpt_functions import get_current_time, tools
from openai import OpenAI
from dotenv import load_dotenv
import os
import json

load_dotenv()
# api_key = os.getenv("OPENAI_API_KEY")  # 환경 변수에서 API 키 가져오기

client = OpenAI(api_key=api_key)  # 오픈AI 클라이언트의 인스턴스 생성

def get_ai_response(messages, tools=None):
    response = client.chat.completions.create(
        model="gpt-4o",  # 응답 생성에 사용할 모델 지정
        messages=messages,  # 대화 기록을 입력으로 전달
        tools=tools,  # 사용 가능한 도구 목록 전달
    )
    return response  # 생성된 응답 내용 반환


In [ ]:
# get_ai_response()함수 실행준비, 151p

messages = [
    {"role": "system", "content": "너는 사용자를 도와주는 상담사야."},  # 초기 시스템 메시지
]

while True:
    user_input = input("사용자\t: ")  # 사용자 입력 받기

    if user_input == "exit":  # 사용자가 대화를 종료하려는지 확인
        break

    messages.append({"role": "user", "content": user_input})  # 사용자 메시지 대화 기록에 추가

    ai_response = get_ai_response(messages, tools=tools)
    ai_message = ai_response.choices[0].message
    print(ai_message)  # ③ gpt에서 반환되는 값을 파악하기 위해 임시로 추가

    tool_calls = ai_message.tool_calls  # AI 응답에 포함된 tool_calls를 가져옵니다.
    if tool_calls:  # tool_calls가 있는 경우
        for tool_call in tool_calls:
            tool_name = tool_call.function.name # 실행해야한다고 판단한 함수명 받기
            tool_call_id = tool_call.id         # tool_call 아이디 받기
            arguments = json.loads(tool_call.function.arguments) # (1) 문자열을 딕셔너리로 변환

            if tool_name == "get_current_time":  # ⑤ 만약 tool_name이 "get_current_time"이라면
                messages.append({
                    "role": "function",  # role을 "function"으로 설정
                    "tool_call_id": tool_call_id,
                    "name": tool_name,
                    "content": get_current_time(timezone=arguments['timezone']),  # 타임존 추가
                })
        messages.append({"role": "system", "content": "이제 주어진 결과를 바탕으로 답변할 차례다."})  # 함수 실행 완료 메시지 추가
        ai_response = get_ai_response(messages, tools=tools) # 다시 GPT 응답 받기
        ai_message = ai_response.choices[0].message

    messages.append(ai_message)  # AI 응답을 대화 기록에 추가하기

    print("AI\t: " + ai_message.content)  # AI 응답 출력


In [ ]:
ai_response

In [ ]:
# 154p
from datetime import datetime
import pytz

def get_current_time(timezone: str = 'Asia/Seoul'):
    tz = pytz.timezone(timezone) # 타임존 설정
    now = datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
    now_timezone = f'{now} {timezone}'
    print(now_timezone)
    return now_timezone


tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_time",
            "description": "해당 타임존의 날짜와 시간을 반환합니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    'timezone': {
                        'type': 'string',
                        'description': '현재 날짜와 시간을 반환할 타임존을 입력하세요. (예: Asia/Seoul)',
                    },
                },
                "required": ['timezone'],
            },
        }
    },
]


if __name__ == '__main__':
    get_current_time('America/New_York')

In [ ]:
# 스트림릿에서 펑션 콜링, 159p~163
from gpt_functions import get_current_time, tools
from openai import OpenAI
from dotenv import load_dotenv
import os
import json
import streamlit as st

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")  # 환경 변수에서 API 키 가져오기

client = OpenAI(api_key=api_key)  # 오픈AI 클라이언트의 인스턴스 생성

def get_ai_response(messages, tools=None):
    response = client.chat.completions.create(
        model="gpt-4o",  # 응답 생성에 사용할 모델 지정
        messages=messages,  # 대화 기록을 입력으로 전달
        tools=tools,  # 사용 가능한 도구 목록 전달
    )
    return response  # 생성된 응답 내용 반환

st.title("💬 Chatbot")

if "messages" not in st.session_state:
    st.session_state["messages"] = [
        {"role": "system", "content": "너는 사용자를 도와주는 상담사야."},  # 초기 시스템 메시지
    ]

for msg in st.session_state.messages:
    if msg["role"] == "assistant" or msg["role"] == "user": # assistant 혹은 user 메시지인 경우만
        st.chat_message(msg["role"]).write(msg["content"])


if user_input := st.chat_input():    # ① 사용자 입력 받기
    st.session_state.messages.append({"role": "user", "content": user_input})  # ① 사용자 메시지를 대화 기록에 추가
    st.chat_message("user").write(user_input)  # ① 사용자 메시지를 브라우저에서도 출력

    ai_response = get_ai_response(st.session_state.messages, tools=tools)
    ai_message = ai_response.choices[0].message
    print(ai_message)  # ③ gpt에서 반환되는 값을 파악하기 위해 임시로 추가

    tool_calls = ai_message.tool_calls  # AI 응답에 포함된 tool_calls를 가져옵니다.
    if tool_calls:  # tool_calls가 있는 경우
        for tool_call in tool_calls:
            tool_name = tool_call.function.name # 실행해야한다고 판단한 함수명 받기
            tool_call_id = tool_call.id         # tool_call 아이디 받기
            arguments = json.loads(tool_call.function.arguments) # (1) 문자열을 딕셔너리로 변환

            if tool_name == "get_current_time":  # ⑤ 만약 tool_name이 "get_current_time"이라면
                st.session_state.messages.append({
                    "role": "function",  # role을 "function"으로 설정
                    "tool_call_id": tool_call_id,
                    "name": tool_name,
                    "content": get_current_time(timezone=arguments['timezone']),  # 타임존 추가
                })
        st.session_state.messages.append({"role": "system", "content": "이제 주어진 결과를 바탕으로 답변할 차례다."})
        ai_response = get_ai_response(st.session_state.messages, tools=tools) # 다시 GPT 응답 받기
        ai_message = ai_response.choices[0].message

    st.session_state.messages.append({
        "role": "assistant",
        "content": ai_message.content
    })  # ③ AI 응답을 대화 기록에 추가합니다.

    print("AI\t: " + ai_message.content)  # AI 응답 출력
    st.chat_message("assistant").write(ai_message.content)  # 브라우저에 메시지 출력

In [ ]:
# GPT와 미국 주식 이야기하기, 164p. 설치 안해도 됨
!pip install yfinance

In [ ]:
# 마이크로소프트 주식 정보, 164p
import yfinance as yf

# Microsoft (MSFT)에 대한 Ticker 객체 생성
msft = yf.Ticker("MSFT")

# Ticker 객체에 대한 정보 출력 (.py에서 실행할 때는 print(msft.info)로 사용)
display(msft.info)

In [ ]:
# 마이크로소프트 주식 정보, 165p
hist = msft.history(period="5d") # 5일간의 주가 데이터를 가져옴
display(hist) # 데이터 출력

In [ ]:
# 마이크로소프트 주식 정보, 165p
msft.recommendations # 추천 정보 출력

In [ ]:
# GPT 함수 정의, 168p
from datetime import datetime
import pytz
import yfinance as yf

def get_current_time(timezone: str = 'Asia/Seoul'):  # 현재 날짜/시간 가져오는 함수
    tz = pytz.timezone(timezone) # 타임존 설정
    now = datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
    now_timezone = f'{now} {timezone}'
    print(now_timezone)
    return now_timezone

def get_yf_stock_info(ticker: str):  # 종목정보 가져오는 함수
    stock = yf.Ticker(ticker)
    info = stock.info
    print(info)
    return str(info)

def get_yf_stock_history(ticker: str, period: str):  # 해당 종목의 주가정보 가져오는 함수
    stock = yf.Ticker(ticker)
    history = stock.history(period=period)
    history_md = history.to_markdown() # 데이터프레임을 마크다운 형식으로 변환
    print(history_md)
    return history_md

def get_yf_stock_recommendations(ticker: str):  # 종목추천 함수
    stock = yf.Ticker(ticker)
    recommendations = stock.recommendations
    recommendations_md = recommendations.to_markdown() # 데이터프레임을 마크다운 형식으로 변환
    print(recommendations_md)
    return recommendations_md


tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_time",
            "description": "해당 타임존의 날짜와 시간을 반환합니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    'timezone': {
                        'type': 'string',
                        'description': '현재 날짜와 시간을 반환할 타임존을 입력하세요. (예: Asia/Seoul)',
                    },
                },
                "required": ['timezone'],
            },
        }
    },
    {  # get_yf_stock_info
        "type": "function",
        "function": {
            "name": "get_yf_stock_info",
            "description": "해당 종목의 Yahoo Finance 정보를 반환합니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    'ticker': {
                        'type': 'string',
                        'description': 'Yahoo Finance 정보를 반환할 종목의 티커를 입력하세요. (예: AAPL)',
                    },
                },
                "required": ['ticker'],
            },
        }
    },
    {  # get_yf_stock_history
        "type": "function",
        "function": {
            "name": "get_yf_stock_history",
            "description": "해당 종목의 Yahoo Finance 주가 정보를 반환합니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    'ticker': {
                        'type': 'string',
                        'description': 'Yahoo Finance 주가 정보를 반환할 종목의 티커를 입력하세요. (예: AAPL)',
                    },
                    'period': {
                        'type': 'string',
                        'description': '주가 정보를 조회할 기간을 입력하세요. (예: 1d, 5d, 1mo, 1y, 5y)',
                    },
                },
                "required": ['ticker', 'period'],
            },
        }
    },
    {  # get_yf_stock_recommendations
        "type": "function",
        "function": {
            "name": "get_yf_stock_recommendations",
            "description": "해당 종목의 Yahoo Finance 추천 정보를 반환합니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    'ticker': {
                        'type': 'string',
                        'description': 'Yahoo Finance 추천 정보를 반환할 종목의 티커를 입력하세요. (예: AAPL)',
                    },
                },
                "required": ['ticker'],
            },
        }
    },
]


if __name__ == '__main__':
    # get_current_time('America/New_York')
    # info = get_yf_stock_info('AAPL')

    get_yf_stock_history('AAPL', '5d')
    print('----')
    get_yf_stock_recommendations('AAPL')


In [ ]:
#
from gpt_functions import get_current_time, tools, get_yf_stock_info, get_yf_stock_history, get_yf_stock_recommendations
from openai import OpenAI
from dotenv import load_dotenv
import os
import json
import streamlit as st

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")  # 환경 변수에서 API 키 가져오기

client = OpenAI(api_key=api_key)  # 오픈AI 클라이언트의 인스턴스 생성

def get_ai_response(messages, tools=None):
    response = client.chat.completions.create(
        model="gpt-4o",  # 응답 생성에 사용할 모델 지정
        messages=messages,  # 대화 기록을 입력으로 전달
        tools=tools,  # 사용 가능한 도구 목록 전달
    )
    return response  # 생성된 응답 내용 반환

st.title("💬 Chatbot")

if "messages" not in st.session_state:
    st.session_state["messages"] = [
        {"role": "system", "content": "너는 사용자를 도와주는 상담사야."},  # 초기 시스템 메시지
    ]

for msg in st.session_state.messages:
    if msg["role"] == "assistant" or msg["role"] == "user": # assistant 혹은 user 메시지인 경우만
        st.chat_message(msg["role"]).write(msg["content"])


if user_input := st.chat_input():    # 사용자 입력 받기
    st.session_state.messages.append({"role": "user", "content": user_input})  # 사용자 메시지를 대화 기록에 추가
    st.chat_message("user").write(user_input)  # 사용자 메시지를 브라우저에서도 출력

    ai_response = get_ai_response(st.session_state.messages, tools=tools)
    ai_message = ai_response.choices[0].message
    print(ai_message)  # gpt에서 반환되는 값을 파악하기 위해 임시로 추가

    tool_calls = ai_message.tool_calls  # AI 응답에 포함된 tool_calls를 가져옵니다.
    if tool_calls:  # tool_calls가 있는 경우
        for tool_call in tool_calls:
            tool_name = tool_call.function.name # 실행해야한다고 판단한 함수명 받기
            tool_call_id = tool_call.id         # tool_call 아이디 받기
            arguments = json.loads(tool_call.function.arguments) # 문자열을 딕셔너리로 변환

            if tool_name == "get_current_time":
                func_result = get_current_time(timezone=arguments['timezone'])
            elif tool_name == "get_yf_stock_info":
                func_result = get_yf_stock_info(ticker=arguments['ticker'])
            elif tool_name == "get_yf_stock_history":  # get_yf_stock_history 함수 호출
                func_result = get_yf_stock_history(
                    ticker=arguments['ticker'],
                    period=arguments['period']
                )
            elif tool_name == "get_yf_stock_recommendations":  # get_yf_stock_recommendations 함수 호출
                func_result = get_yf_stock_recommendations(
                    ticker=arguments['ticker']
                )

            st.session_state.messages.append({
                "role": "function",
                "tool_call_id": tool_call_id,
                "name": tool_name,
                "content": func_result,
            })


        st.session_state.messages.append({"role": "system", "content": "이제 주어진 결과를 바탕으로 답변할 차례다."})
        ai_response = get_ai_response(st.session_state.messages, tools=tools) # 다시 GPT 응답 받기
        ai_message = ai_response.choices[0].message

    st.session_state.messages.append({
        "role": "assistant",
        "content": ai_message.content
    })  # ③ AI 응답을 대화 기록에 추가합니다.

    print("AI\t: " + ai_message.content)  # AI 응답 출력
    st.chat_message("assistant").write(ai_message.content)  # 브라우저에 메시지 출력

## 8장

In [ ]:
# 랭체인을 사용한 챗봇: 싱글턴, 199p
!pip install langchain langchain-openai

In [ ]:
# 랭체인을 사용한 챗봇: 모델생성, 200p
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4o-mini", api_key=api_key)

In [ ]:
# 랭체인을 사용한 챗봇: 메시지보내기, 200p
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="안녕? 나는 홍길동이야.")])

In [ ]:
# 랭체인을 사용한 챗봇: 메시지보내기2, 201p
model.invoke([HumanMessage(content="내 이름이 뭐지?")])

In [ ]:
# 랭체인을 사용한 챗봇: 멀티턴, 202p
# 기존메시지 리스트에 추가하는 랭체인 멀티턴, langchain_multi_turn.py
# from openai import OpenAI  # 주석처리
# from dotenv import load_dotenv
# import os
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage


# load_dotenv()
# api_key = os.getenv("OPENAI_API_KEY")  # 환경 변수에서 API 키 가져오기
# client = OpenAI(api_key=api_key)  # 오픈AI 클라이언트의 인스턴스 생성

# llm = ChatOpenAI(model="gpt-4o")  # ChatOpenAI 클래스의 인스턴스 생성
llm = ChatOpenAI(model="gpt-4o", api_key=api_key)

# def get_ai_response(messages):
#     response = client.chat.completions.create(
#         model="gpt-4o",  # 응답 생성에 사용할 모델 지정
#         temperature=0.9,  # 응답 생성에 사용할 temperature 설정
#         messages=messages,  # 대화 기록을 입력으로 전달
#     )
#     return response.choices[0].message.content  # 생성된 응답의 내용 반환

messages = [
    # {"role": "system", "content": "너는 사용자를 도와주는 상담사야."},  # 초기 시스템 메시지
    SystemMessage("너는 사용자를 도와주는 상담사야."),  # 초기 시스템 메시지
]

while True:
    user_input = input("사용자: ")  # 사용자 입력 받기

    if user_input == "exit":  # ② 사용자가 대화를 종료하려는지 확인인
        break

    messages.append(
        # {"role": "user", "content": user_input} # 주석처리
        HumanMessage(user_input)
    )  # 사용자 메시지를 대화 기록에 추가

    # ai_response = get_ai_response(messages)  # 주석처리
    ai_response = llm.invoke(messages)  # 대화 기록을 기반으로 AI 응답 가져오기
    messages.append(
        # {"role": "assistant", "content": ai_response} # 주석처리
        ai_response
    )  # AI 응답 대화 기록에 추가하기

    print("AI: " + ai_response.content)  # AI 응답 출력


In [ ]:
# 랭체인을 사용한 챗봇: 멀티턴, 204p
# 랭체인의 메시지 히스토리 사용으로 랭체인 멀티턴
from langchain_core.chat_history import InMemoryChatMessageHistory  # 메모리에 대화 기록을 저장하는 클래스
from langchain_core.runnables.history import RunnableWithMessageHistory  # 메시지 기록을 활용해 실행 가능한 래퍼wrapper 클래스
from langchain_openai import ChatOpenAI  # 오픈AI 모델을 사용하는 랭체인 챗봇 클래스
from langchain_core.messages import HumanMessage

# model = ChatOpenAI(model="gpt-4o-mini")
model = ChatOpenAI(model="gpt-4o", api_key=api_key)
# 세션별 대화 기록을 저장할 딕셔너리
store = {}

# 세션 ID에 따라 대화 기록을 가져오는 함수
def get_session_history(session_id: str):
    # 만약 해당 세션 ID가 store에 없으면, 새로 생성해 추가함
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()  # 메모리에 대화 기록을 저장하는 객체 생성
    return store[session_id]  # 해당 세션의 대화 기록을 반환

# 모델 실행 시 대화 기록을 함께 전달하는 래퍼 객체 생성
with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [ ]:
# 메시지 히스토리 연습, 205p, "session_id": "abc2"
config = {"configurable": {"session_id": "abc2"}}  # 세션 ID를 설정하는 config 객체 생성

response = with_message_history.invoke(
    [HumanMessage(content="안녕? 난 홍길동이야.")],
    config=config,
)

print(response.content)

In [ ]:
# 메시지 히스토리 연습, 205p, "session_id": "abc2"
response = with_message_history.invoke(
    [HumanMessage(content="내 이름이 뭐지?")],
    config=config,
)

print(response.content)

In [ ]:
# 메시지 히스토리 연습, 205p, "session_id": "abc3"
config = {"configurable": {"session_id": "abc3"}}

response = with_message_history.invoke(
    [HumanMessage(content="내 이름이 뭐지?")],
    config=config,
)

response.content

In [ ]:
# 메시지 히스토리 연습, 206p, "session_id": "abc2"
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="아까 우리가 무슨 얘기 했지?")],
    config=config,
)

response.content

In [ ]:
# 메시지 히스토리 연습, 207p, "session_id": "abc2"
config = {"configurable": {"session_id": "abc2"}}
for r in with_message_history.stream(
    [HumanMessage(content = "내가 어느 나라 사람인지 맞춰보고, 그 나라의 문화에 대해 말해봐")],
    config=config,
):
    print(r.content, end="|")

In [ ]:
# 8-02, lcel.ipynb
# GPT에게 미녀와 야수 이야기의 미녀 역할 부여하고 대화하기, 208p
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

# model = ChatOpenAI(model="gpt-4o-mini")
model = ChatOpenAI(model="gpt-4o", api_key=api_key)

messages = [
    SystemMessage(content="너는 미녀와 야수에 나오는 미녀야. 그 캐릭터에 맞게 사용자와 대화하라."),
    HumanMessage(content="안녕? 저는 개스톤입니다. 오늘 시간 괜찮으시면 저녁 같이 먹을까요?"),
]

model.invoke(messages)

In [ ]:
# StrOutputParser로 텍스트만 반환하도록 수정하기, 209p
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

result = model.invoke(messages)
parser.invoke(result)

In [ ]:
# Chain 연산자 |를 사용해서 간단하게 수정하기, 210p
chain = model | parser
chain.invoke(messages)

In [ ]:
# 프롬프트 템플릿 이용하기, 211p
from langchain_core.prompts import ChatPromptTemplate

system_template = "너는 {story}에 나오는 {character_a} 역할이다. 그 캐릭터에 맞게 사용자와 대화하라."
human_template = "안녕? 저는 {character_b}입니다. 오늘 시간 괜찮으시면 {activity} 같이 할까요?"

prompt_template = ChatPromptTemplate([
    ("system", system_template),
    ("user", human_template),
])

result = prompt_template.invoke({
    "story": "미녀와 야수",
    "character_a": "미녀",
    "character_b": "야수",
    "activity": "저녁"
})

print(result)

In [ ]:
# 체인으로 구성, 211p
chain = prompt_template | model | parser

chain.invoke({
    "story": "미녀와 야수",
    "character_a": "미녀",
    "character_b": "야수",
    "activity": "저녁"
})

In [ ]:
chain = prompt_template | model | parser

chain.invoke({
    "story": "미녀와 야수",
    "character_a": "미녀",
    "character_b": "개스톤",
    "activity": "저녁"
})

In [ ]:
from typing import Literal
from pydantic import BaseModel, Field

class Adlib(BaseModel):
    """스토리 설정과 사용자 입력에 반응하는 대사를 만드는 클래스"""
    answer: str = Field(description="스토리 설정과 사용자와의 대화 기록에 따라 생성된 대사")
    main_emotion: Literal["기쁨", "분노", "슬픔", "공포", "냉소", "불쾌", "중립"] = Field(description="대사의 주요 감정")
    main_emotion_intensity: float = Field(description="대사의 주요 감정의 강도 (0.0 ~ 1.0)")

structured_llm = model.with_structured_output(Adlib)
adlib_chain = prompt_template | structured_llm

adlib_chain.invoke({
    "story": "미녀와 야수",
    "character_a": "벨",
    "character_b": "개스톤",
    "activity": "저녁"
})

## AI 생성코드

### 음성파일 또는 동영상파일에서 텍스트 추출

In [ ]:
# ================================
# Colab Whisper 음성 → 텍스트 변환
# ================================

# 1. 필수 라이브러리 설치
!pip install -q openai-whisper ffmpeg


In [ ]:
# 2. 구글 드라이브 연동 (선택: 변환된 파일 저장용)
from google.colab import drive
drive.mount('/content/drive')


In [ ]:

# 3. Whisper 불러오기
import whisper

# 4. 모델 로드 (base 권장, 더 빠름 / 정확도 높이려면 'small' 또는 'medium')
model = whisper.load_model("base")

# 5. 오디오 파일 업로드
from google.colab import files
uploaded = files.upload()

# 업로드한 파일 이름 가져오기
import os
audio_file = list(uploaded.keys())[0]

# 6. 음성 파일을 텍스트로 변환
result = model.transcribe(audio_file, language="ko")

# 7. 텍스트 출력
print("\n===== 추출된 텍스트 =====\n")
print(result["text"])


In [ ]:
# 8. txt 파일로 저장
output_path = audio_file.replace(".wav", ".txt").replace(".mp4", ".txt").replace(".mp3", ".txt")
with open(output_path, "w", encoding="utf-8") as f:
    f.write(result["text"])

print(f"\n✅ 변환 완료: {output_path}")

# 9. 다운로드 링크 제공
from google.colab import files
files.download(output_path)


### GEMINI API사용

In [ ]:
import google.generativeai as genai
import os

genai.configure(api_key=userdata.get('GEMINI_API_KEY'))
model = genai.GenerativeModel('gemini-2.0-flash')
response = model.generate_content("인공지능에 대해 자세히 설명해줘.", stream=True)

for no, chunk in enumerate(response, start=1):
  print(f"{no}: {chunk.text}")
  print("="*50)